In [22]:
import re
import os.path as path
from gensim.models.phrases import Phrases
from textblob_de import TextBlobDE as TextBlob

wanted_tags = {'NN', 'NNS', 'NNP', 'JJ'}
stopwords = {'ihr', 'euch'}
pattern = re.compile(r'\W')

In [23]:
def to_keywords(doc):
    letters_only = pattern.sub(' ', doc)
    blob = TextBlob(letters_only)
    keywords = [word.lower() for word, tag in blob.tags
                if word.lower() not in stopwords and tag in wanted_tags]
    return keywords

In [24]:
print(to_keywords('Halle ihr ##Schnarchnasen, ich sag euch jetzt mal was: Apple Google Bosch Ihr seid dreck-mugel alls Huren, ja, dreckige Fotzen. Angela Merkel. Der Islamische Staat.'))

['halle', 'schnarchnasen', 'sag', 'apple', 'google', 'bosch', 'seid', 'dreck', 'mugel', 'huren', 'dreckige', 'fotzen', 'angela', 'merkel', 'islamische', 'staat']
